In [1]:
cd '/content/drive/MyDrive/76_new'

/content/drive/MyDrive/76_new


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import datetime

import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
# from model_attn import RaSCNN

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, initializers, regularizers
import tensorflow.keras.backend as K

# fix CUDNN_STATUS_INTERNAL_ERROR
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# Perform PCA
do_pca = False


In [3]:
def convert_to_one_hot(in_arr):
    in_arr = in_arr.reshape((in_arr.shape[0],))
    in_arr = in_arr.astype(int)
    in_arr_1h = np.zeros((in_arr.size, in_arr.max()+1))
    in_arr_1h[np.arange(in_arr.size),in_arr] = 1
    return in_arr_1h

In [4]:
X_train_valid = np.load("Dataset/X_train_valid.npy").astype(np.float32)
y_train_valid = np.load("Dataset/y_train_valid.npy").astype(np.float32)
person_train_valid = np.load("Dataset/person_train_valid.npy").astype(np.float32)
X_test = np.load("Dataset/X_test.npy").astype(np.float32)
y_test = np.load("Dataset/y_test.npy").astype(np.float32)
person_test = np.load("Dataset/person_test.npy").astype(np.float32)

y_train_valid -= 769
y_test -= 769
# print('y_train_valid:', y_train_valid[:10])
# print('y_test:', y_test[:10])
# X_train_valid, y_train_valid = data_prep(X_train_valid, y_train_valid, 2, 2, True)
# X_test, y_test = data_prep(X_test, y_test, 2, 2, True)

y_train_valid_1h = convert_to_one_hot(y_train_valid)
y_test_1h = convert_to_one_hot(y_test)

In [5]:
print(X_train_valid.shape)
print(y_train_valid_1h.shape)

(2115, 22, 1000)
(2115, 4)


## Normalization

In [6]:
X_train_valid -= np.expand_dims(np.mean(X_train_valid, axis=-1), -1)
X_test -= np.expand_dims(np.mean(X_test, axis=-1), -1)
X_train_valid /= np.expand_dims(np.std(X_train_valid, axis=-1), -1)
X_test /= np.expand_dims(np.std(X_test, axis=-1), -1)

X_train_valid = X_train_valid.transpose((0, 2, 1))

##Model

In [7]:
def RaSCNN(inputshape, outputshape, params=None):
    """
    Spatial summary convolutional neural network augmented with attention
    focused recurrence (Ra-SCNN) by Kostas, Pang, and Rudzics in "Machine
    learning for MEG during speec tasks."
    Adapted from https://github.com/SPOClab-ca/SCNN/blob/master/SCNN/models.py
    """
    ret_seq = True
    att_depth = 4
    attention = 76
    temp_layers = 4
    steps = 2
    temporal = 24
    temp_pool = 20

    lunits = [200, 40]
    activation = keras.activations.selu
    reg = float(0.01)
    dropout = 0.55

    ins = keras.layers.Input(inputshape)
    conv = keras.layers.Reshape((inputshape[0], inputshape[1], 1))(ins)



    ###Spatial Conv
    spatial_filters = 128
    conv = keras.layers.Conv2D(spatial_filters, kernel_size=(1, 11), activation=activation, name='spatial_conv1', kernel_regularizer=tf.keras.regularizers.l2(reg))(conv)
    conv = keras.layers.Conv2D(spatial_filters, kernel_size=(1, 12), activation=activation, name='spatial_conv2', kernel_regularizer=tf.keras.regularizers.l2(reg))(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = keras.layers.SpatialDropout2D(dropout)(conv)



    ###Temporal Conv
    temporal_filters = 64
    conv = keras.layers.Conv2D(temporal_filters, (temporal, 1), activation=activation,
            use_bias=False, name='temporal_conv1',
            kernel_regularizer=tf.keras.regularizers.l2(reg))(conv)
    conv = keras.layers.Conv2D(temporal_filters, (temporal, 1), activation=activation,
            use_bias=False, name='temporal_conv2',
            kernel_regularizer=tf.keras.regularizers.l2(reg))(conv)
    conv = keras.layers.Conv2D(temporal_filters, (temporal, 1), activation=activation,
            use_bias=False, name='temporal_conv3',
            kernel_regularizer=tf.keras.regularizers.l2(reg))(conv)
    conv = keras.layers.Conv2D(temporal_filters, (temporal, 1), activation=activation,
            use_bias=False, name='temporal_conv4',
            kernel_regularizer=tf.keras.regularizers.l2(reg))(conv)

    conv = keras.layers.BatchNormalization()(conv)
    conv = keras.layers.AveragePooling2D((temp_pool, 1,))(conv)
    conv = keras.layers.SpatialDropout2D(dropout)(conv)
    conv = keras.layers.Reshape((45, temporal_filters))(conv)


    lstm = tf.keras.layers.LSTM(units=attention, implementation=2,
                                dropout=dropout, return_sequences=True, return_state=True, kernel_regularizer=tf.keras.regularizers.l2(reg))
    all_states = keras.layers.Bidirectional(lstm)(conv)
    out = all_states[0]
    hidden_states = tf.keras.layers.Concatenate(axis=1)([all_states[1], all_states[3]])
    hidden_states = keras.layers.Reshape((152, 1))(hidden_states)
    attn = tf.matmul(out, hidden_states)
    attn = tf.keras.layers.Softmax()(attn)

    attn_out = tf.keras.layers.Dot(axes=1)([attn, out])


    if ret_seq:
        conv = keras.layers.Flatten()(attn_out)
    outs = conv

    outs = keras.layers.Dense(outputshape, activation='softmax')(outs)

    return keras.models.Model(ins, outs)

In [8]:
model = RaSCNN(inputshape=(1000, 22), outputshape=4)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1000, 22)]   0           []                               
                                                                                                  
 reshape (Reshape)              (None, 1000, 22, 1)  0           ['input_1[0][0]']                
                                                                                                  
 spatial_conv1 (Conv2D)         (None, 1000, 12, 12  1536        ['reshape[0][0]']                
                                8)                                                                
                                                                                                  
 spatial_conv2 (Conv2D)         (None, 1000, 1, 128  196736      ['spatial_conv1[0][0]']      

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=[tf.keras.metrics.CategoricalAccuracy()])

callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        patience=10,
        factor=0.5,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath= 'model_{epoch:03d}.ckpt',
        save_best_only=False,
        save_weights_only=True,
        verbose=1,
        monitor='val_categorical_accuracy',
        # monitor='val_loss'
    ),
]

In [ ]:
loss_hist = model.fit(
        X_train_valid, y_train_valid_1h,
        validation_split=0.2,
        batch_size=64,
        epochs=300,
        verbose=2,
        callbacks=callbacks,

    )

Epoch 1/300

Epoch 1: saving model to model_001.ckpt
27/27 - 15s - loss: 10.5182 - categorical_accuracy: 0.2843 - val_loss: 10.3328 - val_categorical_accuracy: 0.2530 - lr: 1.0000e-04 - 15s/epoch - 549ms/step
Epoch 2/300

Epoch 2: saving model to model_002.ckpt
27/27 - 3s - loss: 10.1519 - categorical_accuracy: 0.2600 - val_loss: 10.5828 - val_categorical_accuracy: 0.2530 - lr: 1.0000e-04 - 3s/epoch - 123ms/step
Epoch 3/300

Epoch 3: saving model to model_003.ckpt
27/27 - 3s - loss: 9.7877 - categorical_accuracy: 0.2831 - val_loss: 9.2458 - val_categorical_accuracy: 0.2553 - lr: 1.0000e-04 - 3s/epoch - 121ms/step
Epoch 4/300

Epoch 4: saving model to model_004.ckpt
27/27 - 3s - loss: 9.4509 - categorical_accuracy: 0.2719 - val_loss: 8.6088 - val_categorical_accuracy: 0.2530 - lr: 1.0000e-04 - 3s/epoch - 124ms/step
Epoch 5/300

Epoch 5: saving model to model_005.ckpt
27/27 - 3s - loss: 9.1189 - categorical_accuracy: 0.2908 - val_loss: 8.4848 - val_categorical_accuracy: 0.2695 - lr: 1.00

In [ ]:
import numpy as np
max_idx = np.argmax(acc_list) + 1
print(max_idx)
best_filename = 'model_{:03d}.ckpt'.format(max_idx)
model.load_weights(best_filename)
print('\nTest accuracy')
loss, acc = model.evaluate(X_test.transpose((0, 2, 1)), y_test_1h)
print("Test accuracy: {:5.2f}%".format(100 * acc))

191

Test accuracy
14/14 [==============================] - 0s 19ms/step - loss: 2.7562 - categorical_accuracy: 0.7088
Test accuracy: 70.88%
